### Finding Prime Numbers

In [4]:
n = 500000
allnumbers = sc.parallelize(xrange(2, n), 8).cache()
composite = allnumbers.flatMap(lambda x: xrange(x*2, n,  )).repartition(8)
prime = allnumbers.subtract(composite)
print prime.take(10)

[17, 401537, 462641, 97, 47137, 113, 43649, 467009, 193, 488833]


In [2]:
# Find the number of elements in each parttion
def partitionsize(it): 
    s = 0
    for i in it:
        s += 1
    yield s

print allnumbers.mapPartitions(partitionsize).collect()
print composite.mapPartitions(partitionsize).collect()
print prime.mapPartitions(partitionsize).collect()
print prime.glom().collect()[1][0:4]

[62499, 62500, 62500, 62500, 62499, 62500, 62500, 62500]
[5216986, 254759, 104166, 62499, 0, 0, 0, 0]
[0, 5169, 1, 5219, 0, 5206, 0, 5189, 0, 5165, 0, 5199, 0, 5191, 0, 5199]
[17, 401537, 462641, 97]


### Data Partitioning

In [4]:
data = [8, 96, 240, 400, 401, 800]
rdd = sc.parallelize(zip(data, data),4)
print rdd.partitioner
print rdd.glom().collect()
rdd = rdd.reduceByKey(lambda x,y: x+y)
print rdd.glom().collect()
print rdd.partitioner.partitionFunc
rdd = rdd.sortByKey()
print rdd.glom().collect()
print rdd.partitioner.partitionFunc

None
[[(8, 8)], [(96, 96), (240, 240)], [(400, 400)], [(401, 401), (800, 800)]]
[[(8, 8), (96, 96), (400, 400), (240, 240), (800, 800)], [(401, 401)], [], []]
<function portable_hash at 0x7f5634d03f50>
[[(8, 8), (96, 96)], [(240, 240), (400, 400)], [(401, 401)], [(800, 800)]]
<function rangePartitioner at 0x7f56347bf320>


In [11]:
a = sc.parallelize(zip(range(10000), range(10000)), 8)
b = sc.parallelize(zip(range(10000), range(10000)), 8)
print a.partitioner
a = a.reduceByKey(lambda x,y: x+y)
print a.partitioner.partitionFunc
b = b.reduceByKey(lambda x,y: x+y)
c = a.join(b)
print c.getNumPartitions()
print c.partitioner.partitionFunc
print c.glom().first()[0:4]

None
<function portable_hash at 0x7f5634d03f50>
8
<function portable_hash at 0x7f5634d03f50>
[(0, (0, 0)), (2048, (2048, 2048)), (1432, (1432, 1432)), (2056, (2056, 2056))]


In [1]:
# A 'real' example from SF Express
# Prepare three relational tables

from pyspark.sql.functions import *

num_waybills = 1000
num_customers = 100

rdd = sc.parallelize((i, ) for i in xrange(num_waybills))
waybills = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                             floor(rand()*num_customers).alias('customer')) \
                .groupBy('waybill').max('customer').withColumnRenamed('max(customer)','customer')\
                .cache()
waybills.show()
print waybills.count()

rdd = sc.parallelize((i, i) for i in xrange(num_customers))
customers = spark.createDataFrame(rdd, ['customer', 'phone']).cache()
customers.show()
print customers.count()

rdd = sc.parallelize((i, ) for i in xrange(num_waybills))
waybill_status = spark.createDataFrame(rdd).select(floor(rand()*num_waybills).alias('waybill'), 
                                                   floor(rand()*10).alias('version')) \
                      .groupBy('waybill').max('version').cache()
waybill_status.show()
print waybill_status.count()

+-------+--------+
|waybill|customer|
+-------+--------+
|     29|      94|
|    474|      80|
|    964|      47|
|    558|      87|
|    191|      81|
|    418|      59|
|    270|      30|
|    222|      26|
|    293|      12|
|    730|      65|
|    938|      79|
|    442|      90|
|    367|      34|
|    278|      40|
|    705|      47|
|    243|      78|
|    296|      92|
|    926|      37|
|     54|      16|
|    847|      90|
+-------+--------+
only showing top 20 rows

628
+--------+-----+
|customer|phone|
+--------+-----+
|       0|    0|
|       1|    1|
|       2|    2|
|       3|    3|
|       4|    4|
|       5|    5|
|       6|    6|
|       7|    7|
|       8|    8|
|       9|    9|
|      10|   10|
|      11|   11|
|      12|   12|
|      13|   13|
|      14|   14|
|      15|   15|
|      16|   16|
|      17|   17|
|      18|   18|
|      19|   19|
+--------+-----+
only showing top 20 rows

100
+-------+------------+
|waybill|max(version)|
+-------+------------+
|     2

In [3]:
# We want to join 3 tables together.
# Knowing how each table is partitioned helps optimize the join order.

waybills.join(customers, 'customer').join(waybill_status, 'waybill').show()
# waybills.join(waybill_status, 'waybill').join(customers, 'customer').show()

+-------+--------+-----+------------+
|waybill|customer|phone|max(version)|
+-------+--------+-----+------------+
|    474|      80|   80|           3|
|    964|      47|   47|           5|
|    191|      81|   81|           2|
|    222|      26|   26|           4|
|    293|      12|   12|           7|
|    442|      90|   90|           7|
|    278|      40|   40|           8|
|    705|      47|   47|           2|
|    243|      78|   78|           1|
|    926|      37|   37|           6|
|     54|      16|   16|           8|
|    847|      90|   90|           8|
|    348|      42|   42|           7|
|    502|      60|   60|           8|
|    113|      74|   74|           2|
|    155|      99|   99|           3|
|    385|      44|   44|           5|
|    241|      79|   79|           7|
|    414|      15|   15|           0|
|    196|       2|    2|           3|
+-------+--------+-----+------------+
only showing top 20 rows



In [9]:
def partitionsize(it): yield len(list(it))
    
n = 40000

def f(x):
    return x / (n/8)

data1 = range(0, n, 16) + range(0, n, 16)
data2 = range(0, n, 8)
rdd1 = sc.parallelize(zip(data1, data2), 8)
rdd1 = rdd1.reduceByKey(lambda x,y: x+y)
print rdd1.mapPartitions(partitionsize).collect()
rdd1 = rdd1.partitionBy(8, f)
rdd2 = rdd1.reduceByKey(lambda x,y: x+y, partitionFunc=f)
#rdd2 = rdd1.reduceByKey(lambda x,y: x+y)
rdd2.mapPartitions(partitionsize).collect()

[2500, 0, 0, 0, 0, 0, 0, 0]


[313, 312, 313, 312, 313, 312, 313, 312]

### Partitioning in DataFrames

In [5]:
data1 = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4]
data2 = [2, 2, 3, 4, 5, 3, 1, 1, 2, 3]
df = spark.createDataFrame(zip(data1, data2))
print df.rdd.getNumPartitions()
print df.rdd.glom().collect()

8
[[Row(_1=1, _2=2)], [Row(_1=1, _2=2)], [Row(_1=1, _2=3)], [Row(_1=2, _2=4), Row(_1=2, _2=5)], [Row(_1=2, _2=3)], [Row(_1=3, _2=1)], [Row(_1=3, _2=1)], [Row(_1=3, _2=2), Row(_1=4, _2=3)]]


In [6]:
df1 = df.repartition(6, df._2)
print df1.rdd.glom().collect()
df1.show()

[[], [], [Row(_1=1, _2=2), Row(_1=1, _2=2), Row(_1=2, _2=4), Row(_1=2, _2=5), Row(_1=3, _2=2)], [Row(_1=1, _2=3), Row(_1=2, _2=3), Row(_1=4, _2=3)], [], [Row(_1=3, _2=1), Row(_1=3, _2=1)]]
+---+---+
| _1| _2|
+---+---+
|  1|  2|
|  1|  2|
|  2|  4|
|  2|  5|
|  3|  2|
|  1|  3|
|  2|  3|
|  4|  3|
|  3|  1|
|  3|  1|
+---+---+



### Threading

In [3]:
import threading
import random

partitions = 8
n = 500000 * partitions

# use different seeds in different threads and different partitions
# a bit ugly, since mapPartitionsWithIndex takes a function with only index
# and it as parameters
def f1(index, it):
    random.seed(index + 987231)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f2(index, it):
    random.seed(index + 987232)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

def f3(index, it):
    random.seed(index + 987233)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f4(index, it):
    random.seed(index + 987234)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0
    
def f5(index, it):
    random.seed(index + 987245)
    for i in it:
        x = random.random() * 2 - 1
        y = random.random() * 2 - 1
        yield 1 if x ** 2 + y ** 2 < 1 else 0

f = [f1, f2, f3, f4, f5]
    
# the function executed in each thread/job
def dojob(i):
    count = sc.parallelize(xrange(1, n + 1), partitions) \
              .mapPartitionsWithIndex(f[i]).reduce(lambda a,b: a+b)
    print "Worker", i, "reports: Pi is roughly", 4.0 * count / n

# create and execute the threads
threads = []
for i in range(5):
    t = threading.Thread(target=dojob, args=(i,))
    threads += [t]
    t.start()

# wait for all threads to complete
for t in threads:
    t.join()    
'''
for i in range(5):
    dojob(i)
'''

Worker 3 reports: Pi is roughly 3.142212
Worker 4 reports: Pi is roughly 3.142006
Worker 2 reports: Pi is roughly 3.142872
Worker 0 reports: Pi is roughly 3.141997
Worker 1 reports: Pi is roughly 3.142233



for i in range(5):
    dojob(i)
